In [204]:
import numpy as np

In [205]:
data  = np.random.rand(6,6)*1000 # some random data

In [206]:
P = np.array([
             [.2, .3, .5],
             [.3, .4, .3],
             [.5, .3, .2]
             ])

### 3.1 Connectivity

In [207]:
def RBF(x, y, alpha=1):
    """ Gaussian kernel function for two vectors x and y. With neighborhood distance alpha.
    
    Arguments:
        - x : First vector
        - y : Second vector
        - alpha: neighborhood factor. *Note bigger alpha increases RBF value --> bigger neighborhood
    
    return: Real num in [0, 1]
    """
    return np.exp(-np.linalg.norm(np.array(x) - np.array(y))**2 / alpha)

In [208]:
# calculation check
assert((RBF([0,0], [1,1], 1) - np.exp(-2)) < 0.0001)
assert((RBF([0,0], [0,0], 1) - 1 < 0.0001))

In [209]:
def kernelize(data, alpha=1, K=RBF):
    """ Given some data convert it to a kernel matrix using some given kernel.
    
    Arguments:
        data : Data matrix (N, C)
        k: Kernel function
        alpha: neighborhood factor. *Note bigger alpha increases kernel function value.
    """
    # TODO n^2 should be n^2/2 :]
    
    kernel_matrix = []
    for d_i in data:
        row = [K(d_i, d_j, alpha=1) for d_j in data]
        kernel_matrix.append(row)
        
    return np.array(kernel_matrix)

In [ ]:
kernel_matrix = kernelize(X, .5, RBF)
kernel_matrix

In [ ]:
# Add parameter t for the time
def diffusion_matrix_RBF(data, m, alpha=1):
    """ Comptues the diffusion matrix for a given data set. Reducing the dimensionality of 
    each data point to size m.
    
    Arguments:
        data: (N, C) matrix of data points. Each point is a row vector.
        m: New reeduced dimension size of matrix. m <= C.
        alpha: Neighborhood size for RBF kernel
    """
    # TODO enfor 1 <= m <= C
    diffusion_matrix = []
    
    # 1. Define a kernel K    
    kernel_matrix = kernelize(data, alpha, RBF)
    
    # 2. Create diffusion matrix (P - the row normalized kernel)
    P = kernel_matrix / np.sum(kernel_matrix, axis=1).reshape(-1, 1)
    
    
    # 3. Calculate the eigenvectors of the diffusion matrix.
    eigen_vals, eigen_vecs = np.linalg.eig(P)
    eigen_vals = eigen_vals**2
    for i in range(len(data)):
        a = (eigen_vals * eigen_vecs[i , m])
        row = [diffusion_dist2(a, (eigen_vals * eigen_vecs[j , m])) for j in range(m)]
        diffusion_matrix.append(row)    
    
    
    # 4. Map to the d-dimensional diffusion space at time t, 
    #    using the d dominant eigenvectors and -values as

    return np.array(diffusion_matrix)


### 3.3 Diffusion Distance

$
\begin{align}
D_t(X_i, X_j)^2 &= \sum_{u\in{}X}\mid{}p_t(X_i, u) - p_t(X_j, u)\mid{}^2 \\
                &= \sum_{u\in{}X}\mid{}p_{ik}^t - p_{kj}^t\mid{}^2
\end{align}
$

In [ ]:
# Note theres no square in the diffusion distance
def diffusion_dist2(i, j):
    """ Computes the diffusion distance between two vectors X_i and X_j.
    
    Arguments:
        i: first vector
        j: second vector
    """
    return np.linalg.norm(i - j)**2

In [ ]:
# Note theres no square in the diffusion distance
def diffusion_dist(i, j, P):
    """ Computes the diffusion distance between two vectors X_i and X_j.
    
    Arguments:
        i: index for the first data point
        j: index for the second data point
        P: Probability matrix P[i, j] is the probability to go from i to j.
    """
    return np.linalg.norm(P[i, :] - P[j, :])**2

In [ ]:
# P_1 = [.3, .4, .3]
# P_2 = [.5, .3, .2]
assert(np.abs(diffusion_dist(0,1, P) == (.2**2 + .1**2 + .1**2)))
print(diffusion_dist(0,2, P) )

### 3.4 Diffusion Map

In [ ]:
P**14

In [ ]:
# Define Y = P^T
Y = np.transpose(P)

# Get eigen vectors and values of P
eigen_vals, eigen_vecs = np.linalg.eig(P)
Y

### TODO

1. Notice eigen_vecs.. the first column if the first eigen vector (so eigen_vecs[0, :] is first element of each eigen vector
2. the eigen_vals and eigen_vecs are not sorted in increasing e_val order

In [ ]:
eigen_vecs[0 , :]

In [ ]:
eigen_vecs

In [ ]:
a = (eigen_vals * eigen_vecs[0 , :])
b = (eigen_vals * eigen_vecs[2 , :])
print(a)
print(b)

In [ ]:
diffusion_dist2(a, b)

# \#4 Visualizing Different Dimensionality Reducers

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt

# Load MNIST dataset
(x_train_im, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(len(x_train_im),-1)
x_test = x_test.reshape(len(x_test),-1)


In [ ]:
plt.imshow(x_train_im[2])
plt.show()

In [200]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 2)

## 4.2 Iris Dataset w/ PCA and LDA

In [ ]:
x_train = x_train[:500]
y_train = y_train[:500]
X = x_train
y = y_train

X_r3 = diffusion_matrix_RBF(X, 2, alpha=.5)*1000

In [ ]:
print(__doc__)

import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

iris = datasets.load_iris()

X = iris.data
y = iris.target

target_names = iris.target_names

pca = PCA(n_components=2)
X_r = pca.fit(X).transform(X)

lda = LinearDiscriminantAnalysis(n_components=2)
X_r2 = lda.fit(X, y).transform(X)

# Percentage of variance explained for each components
print('explained variance ratio (first two components): %s'
      % str(pca.explained_variance_ratio_))

plt.figure()
colors = ['navy', 'turquoise', 'darkorange']
lw = 2

for color, i, target_name in zip(colors, [0, 1, 2], target_names):
    plt.scatter(X_r[y == i, 0], X_r[y == i, 1], color=color, alpha=.8, lw=lw,
                label=target_name)
plt.legend(loc='best', shadow=False, scatterpoints=1)
plt.title('PCA of IRIS dataset')

plt.figure()
for color, i, target_name in zip(colors, [0, 1, 2], target_names):
    plt.scatter(X_r2[y == i, 0], X_r2[y == i, 1], alpha=.8, color=color,
                label=target_name)
plt.legend(loc='best', shadow=False, scatterpoints=1)
plt.title('LDA of IRIS dataset')


plt.figure()
for color, i, target_name in zip(colors, [0, 1, 2], target_names):
    plt.scatter(X_r3[y == i, 0], X_r3[y == i, 1], alpha=.8, color=color,
                label=target_name)
plt.legend(loc='best', shadow=False, scatterpoints=1)
plt.title('Diffusion Map of IRIS dataset')

plt.show()

## 4.3 Iris Dataset w/ Diffusion 

In [ ]:
print(__doc__)

import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

iris = datasets.load_iris()

X = x_train
y = y_train

target_names = iris.target_names

pca = PCA(n_components=2)
X_r = pca.fit(X).transform(X)

lda = LinearDiscriminantAnalysis(n_components=2)
X_r2 = lda.fit(X, y).transform(X)

# Percentage of variance explained for each components
print('explained variance ratio (first two components): %s'
      % str(pca.explained_variance_ratio_))

plt.figure()
colors = ['navy', 'turquoise', 'darkorange', 'red', 'blue', 'green', 'orange', 'black', 'purple', 'pink']
lw = 2

for color, i, target_name in zip(colors, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], target_names):
    plt.scatter(X_r[y == i, 0], X_r[y == i, 1], color=color, alpha=.8, lw=lw,
                label=target_name)
plt.legend(loc='best', shadow=False, scatterpoints=1)
plt.title('PCA of IRIS dataset')

plt.figure()
for color, i, target_name in zip(colors, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], target_names):
    plt.scatter(X_r2[y == i, 0], X_r2[y == i, 1], alpha=.8, color=color,
                label=target_name)
plt.legend(loc='best', shadow=False, scatterpoints=1)
plt.title('LDA of IRIS dataset')


plt.figure()
for color, i, target_name in zip(colors, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], target_names):
    plt.scatter(X_r3[y == i, 0], X_r3[y == i, 1], alpha=.8, color=color,
                label=target_name)
plt.legend(loc='best', shadow=False, scatterpoints=1)
plt.title('Diffusion Map of IRIS dataset')

plt.show()